In [1]:
import skimage.io as skio
import skimage
import glob
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score
import numpy as np
from bench_filter import Bench_Filter
from sklearn.model_selection import train_test_split
import random
import skimage.morphology as skm
from tools import *

In [2]:
from multiprocessing import cpu_count
cpus = cpu_count()
print(f'Number of CPU cores available : {cpus}')

Number of CPU cores available : 8


In [3]:
DATASET = ['Fluo-N2DL-HeLa','PhC-C2DH-U373']
idx = 1

IM_PATH = sorted(glob.glob(f'data/{DATASET[idx]}/IMG_TRAIN/*.tif'))
TG_PATH = sorted(glob.glob(f'data/{DATASET[idx]}/TARGET_TRAIN/*.tif'))

In [4]:
X_train, y_train = load_img_tg(IM_PATH, TG_PATH)
len(X_train)

184

In [10]:
X_train_cropped, y_train_cropped = [], []
#dimension of patchs = M*N
M = 70
N = 110
for k in range(len(X_train)):
    im = X_train[k]
    tg = y_train[k]
    im_tiles = [im[x:x+M, y:y+N] for x in range(0, im.shape[0], M) for y in range(0, im.shape[1], N)]
    tg_tiles = [tg[x:x+M, y:y+N] for x in range(0, im.shape[0], M) for y in range(0, im.shape[1], N)]
    scores = []
    for i in range(len(im_tiles)):
        scores.append(np.sum(im_tiles[i] > 0.05))
    idx = np.argmax(scores)
    X_train_cropped.append(im_tiles[idx])
    y_train_cropped.append(tg_tiles[idx])
    print(im_tiles[idx].shape)
print(len(X_train_cropped))

(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)
(70, 110)


In [6]:
BF = Bench_Filter(n_estimators=20, cpus=cpus-1) #Leave 1 cpu to avoid computer crash
BF.add_identity()
BF.add_gaussian(np.sqrt(2)/2)
BF.add_gaussian(np.sqrt(2))
BF.add_gaussian(np.sqrt(2)*2) #Progression en sqrt(2)
BF.add_median(skm.disk(3))
BF.add_median()
BF.add_sobel()
BF.add_farid()
BF.add_prewitt()
BF.add_roberts()
BF.add_scharr()

In [7]:
BF.fit(X_train_cropped, y_train_cropped)

Applying filters to dataset: 184it [00:01, 144.28it/s]
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.


building tree 1 of 20building tree 2 of 20

building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=7)]: Done  18 out of  20 | elapsed:   29.3s remaining:    3.3s
[Parallel(n_jobs=7)]: Done  20 out of  20 | elapsed:   30.7s finished


In [8]:
IM_TEST_PATH = sorted(glob.glob(f'data/{DATASET[idx]}/IMG_TEST/*.tif'))
TG_TEST_PATH = sorted(glob.glob(f'data/{DATASET[idx]}/TARGET_TEST/*.tif'))

X_test, y_test = load_img_tg(IM_TEST_PATH, TG_TEST_PATH)

In [9]:
y_pred = BF.predict(X_test)

Applying filters to dataset: 0it [00:00, ?it/s]


ValueError: could not broadcast input array from shape (770000,11) into shape (7700,11)

In [ ]:
results = []
for i, pred in enumerate(y_pred):
    results.append(jaccard_score(y_test[i],pred,average='micro'))
plt.hist(results,bins=100,range=(0,1))
plt.title(f'IoU of random forest on {DATASET[idx]}')
plt.xlabel('IoU score')
plt.ylabel('Frequency')
plt.savefig(f'output/rf_performance_dataset_{idx}.png')

In [ ]:
n_im = random.randrange(len(y_test))
target = y_test[n_im]
seg = skimage.img_as_bool(BF.predict([X_test[n_im]])[0])
score = jaccard_score(target,seg,average='micro')

fig = plot_pred_with_target(target,seg,score)
fig.savefig(f'output/inference_rf_dataset_{idx}.png')

In [ ]:
score = 1
n_im = 0
while score > 0.5 and n_im < len(y_test) :
    target = y_test[n_im]
    seg = skimage.img_as_bool(BF.predict([X_test[n_im]])[0])
    score = jaccard_score(target,seg,average='micro')
    n_im += 1
    
plot_pred_with_target(target,seg,score)
    